## Preparation

In [ ]:
import geopandas as gpd
import pandas as pd
import os
import datetime
import numpy as np
from funcs.charts import daily_charts, daily_sum_charts, season_ET_bar_charts

In [ ]:
# read inputs from spreadsheet
working_dr = os.getcwd()
data_dr = os.path.join(working_dr, 'data')
inputs_dr = os.path.join(working_dr, 'inputs.csv')
inputs_df = pd.read_csv(inputs_dr)

In [ ]:
# choose a season
# by index
index = 0
row = inputs_df.iloc[index]
# # by name
# name = '2020_2021_wheat'
# row = inputs_df.loc[inputs_df['season'] == name].iloc[0]
season = row['season']
project_shp_location = row['project_shp']
field_shp_location = row['field_shp']
start_date = row['start_date']
end_date = row['end_date']
field_shp = gpd.read_file(field_shp_location) #read shapefile
project_shp = gpd.read_file(project_shp_location) #read shapefile
bbox = project_shp.total_bounds.tolist()

In [ ]:
shp_name = os.path.basename(project_shp_location).split('.')[0]
project_dr = os.path.join(working_dr, 'data', shp_name)
season_dr = os.path.join(working_dr, 'data', shp_name, f'season_{season}')
et_folder = os.path.join(season_dr, 'ET')
ndvi_folder = os.path.join(season_dr, 'ndvi')
rgb_folder = os.path.join(season_dr, 'rgb')

In [ ]:
# dataframes
daily_ET_df = pd.read_csv(os.path.join(et_folder, "datasheets", "daily_ET.csv"))
daily_ET_df['date'] = pd.to_datetime(daily_ET_df['date'], format='%Y-%m-%d')
monthly_ET_df = pd.read_csv(os.path.join(et_folder, "datasheets", "monthly_ET.csv"))
monthly_ET_df['Month-Year'] = pd.to_datetime(monthly_ET_df['Month-Year'], format='%Y-%m')
ETa_df = pd.read_csv(os.path.join(et_folder, "datasheets", "ETa.csv"))
kc_df = pd.read_csv(os.path.join(et_folder, "datasheets", "kc.csv"))
daily_ndvi_df = pd.read_csv(os.path.join(ndvi_folder, "datasheets", "daily_ndvi.csv"))
daily_ndvi_df['date'] = pd.to_datetime(daily_ndvi_df['date'], format='%Y-%m-%d')
monthly_ndvi_df = pd.read_csv(os.path.join(ndvi_folder, "datasheets", "monthly_ndvi.csv"))
monthly_ndvi_df['Month-Year'] = pd.to_datetime(monthly_ndvi_df['Month-Year'], format='%Y-%m')
ndvi_df = pd.read_csv(os.path.join(ndvi_folder, "datasheets", "ndvi.csv"))

## charts

In [ ]:
# evapotranspiration charts
daily_charts(daily_ET_df, ETa_df, 'ETa', os.path.join(et_folder,'charts'), 'avarage Daily ETa mm day')
daily_sum_charts(daily_ET_df, 'ETa', os.path.join(et_folder,'charts'), 'Daily ETa mm (sum)')
season_ET_bar_charts(monthly_ET_df, os.path.join(et_folder,'charts'))

In [ ]:
# NDVI charts
daily_charts(daily_ndvi_df, ndvi_df, 'ndvi', os.path.join(ndvi_folder,'charts'), 'Daily NDVI')

## Images layouts

In [ ]:
from funcs.layout import Layouts
layouts = Layouts()

In [ ]:
# days ETa layout
layouts.days_images_layout(os.path.join(et_folder, 'tifs', 'ETa'), os.path.join(et_folder, 'layouts'), field_shp, bbox, 'Daily ETa mm day', 'ETa', 0, 7, True)

In [ ]:
# days NDVI layout
layouts.days_images_layout(os.path.join(ndvi_folder, 'tifs', 'ndvi'), os.path.join(ndvi_folder, 'layouts'), field_shp, bbox, 'daily NDVI', 'ndvi', 0 , 0.8, True)

In [ ]:
# season ETa layout
layouts.season_layout(os.path.join(et_folder, 'tifs'), 'ETa', 'ETa mm', os.path.join(et_folder, 'layouts'), bbox, field_shp, 'season ETa', 200, 700)

In [ ]:
# season ndvi layout
layouts.season_layout(os.path.join(ndvi_folder, 'tifs'), 'ndvi', 'NDVI', os.path.join(ndvi_folder, 'layouts'), bbox, field_shp,'season NDVI', 15, 60)